In [2]:
!pip install -q git+https://github.com/openai/whisper.git
!pip install -q gradio
!pip install -q git+https://github.com/huggingface/transformers

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 47.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 34.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 56.4 MB/s eta

In [3]:
# -*- coding: utf-8 -*-
"""OpenAI Whisper from Hugging Face Transformers with Microsoft PHI 3 Integration"""

import gradio as gr
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
import torch
from huggingface_hub import InferenceClient
import os

# Initialize the InferenceClient for PHI 3
client = InferenceClient(
    "microsoft/phi-3",  # Update this to the correct model name for PHI 3
    token=os.getenv("HF_API_TOKEN")
)

# Check if a GPU is available and use it if possible
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Initialize the Whisper pipeline
whisper = pipeline('automatic-speech-recognition', model='openai/whisper-tiny', device=0 if device == 'cuda' else -1)

# Load Microsoft PHI 3 model and tokenizer (Assuming same method as PHI 1.5)
instructions = os.getenv("INST")


def query_phi(prompt):
    response = ""  # Initialize an empty string to store the response
    for message in client.chat_completion(
      messages=[{"role": "user", "content": f"{instructions}\n{prompt}"}],
      max_tokens=500,
      stream=True,
    ):
        response += message.choices[0].delta.content  # Append each message to the response
    return response  # Return the accumulated response after the loop


def transcribe_and_query(audio):
    # Transcribe the audio file
    transcription = whisper(audio)["text"]
    transcription = "Prompt : " + transcription
    # Query Microsoft PHI 3 with the transcribed text
    phi_response = query_phi(transcription)

    return transcription, phi_response

# Create Gradio interface
iface = gr.Interface(
    fn=transcribe_and_query,
    inputs=gr.Audio(type="filepath"),
    outputs=["text", "text"],
    title="Scam Call detector with BEEP",
    description="Upload your recorded call to see if it is a scam or not. /n Stay Safe, Stay Secure."
)

# Launch the interface
iface.launch()


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.98k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/151M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.75k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

...

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

Device set to use cuda:0


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://afaa2b8f923d260dea.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
# -*- coding: utf-8 -*-
"""OpenAI Whisper from Hugging Face Transformers with Microsoft PHI 3 Integration"""

import torch
from huggingface_hub import InferenceClient
import os
import pickle

# Initialize the InferenceClient for PHI 3
client = InferenceClient(
    "microsoft/phi-3",  # Ensure this is the correct model name for PHI 3
    token=os.getenv("HF_API_TOKEN")
)

# Check if a GPU is available and use it if possible
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Initialize the Whisper pipeline
from transformers import pipeline
whisper = pipeline('automatic-speech-recognition', model='openai/whisper-tiny', device=0 if device == 'cuda' else -1)

# Load Microsoft PHI 3 model and tokenizer
instructions = os.getenv("INST")

def query_phi(prompt):
    response = ""  # Initialize an empty string to store the response
    for message in client.chat_completion(
      messages=[{"role": "user", "content": f"{instructions}\n{prompt}"}],
      max_tokens=500,
      stream=True,
    ):
        response += message.choices[0].delta.content  # Append each message to the response
    return response  # Return the accumulated response after the loop

def transcribe_and_query(audio):
    # Transcribe the audio file
    transcription = whisper(audio)["text"]
    transcription = "Prompt : " + transcription
    # Query Microsoft PHI 3 with the transcribed text
    phi_response = query_phi(transcription)

    return transcription, phi_response

# Save models and functions
def save_models():
    # Save the Whisper model
    with open('whisper_model.pkl', 'wb') as f:
        pickle.dump(whisper, f)

    # Save the PHI 3 client and query function
    with open('phi3_client.pkl', 'wb') as f:
        pickle.dump(client, f)

    # Save the instructions if required
    with open('instructions.txt', 'w') as f:
        f.write(instructions)

    print("Models and components saved.")

# Call this function to save the model and other components
save_models()

# Example for loading and using saved models:

# Load saved Whisper model
with open('whisper_model.pkl', 'rb') as f:
    whisper_loaded = pickle.load(f)

# Load saved PHI 3 client
with open('phi3_client.pkl', 'rb') as f:
    phi3_client_loaded = pickle.load(f)

# Load saved instructions
with open('instructions.txt', 'r') as f:
    instructions_loaded = f.read()

# Now you can use these saved models to transcribe audio and query PHI 3 for detection:
audio_file = 'path_to_audio.wav'  # Replace with your audio file path
transcription, phi_response = transcribe_and_query(audio_file)
print("Transcription:", transcription)
print("PHI 3 Response:", phi_response)
